In [1]:
# change the directory
import os 
from dotenv import load_dotenv
from typing import TypedDict, List, Annotated
from operator import add

# import relevant packages
os.chdir(os.getenv('WORKSPACE_DIRECTORY'))
from agents.base_llm import OpenAILLMs
from agents.master import MasterWorkflow
from agents.nurse import NurseLLM
from agents.diagnosis import DiagnosisLLM

from utils.vector_database import PDFVectorDatabase
from utils.ddg_search import DDGSearch

# from agents.diagnosis_team.diagnosis_coordinator import DiagnosticCoordinator
# from agents.diagnosis_team.generalist import GeneralistLLM

#### Mocked Data

In [2]:
class Report(TypedDict):
    """
    Defines the state for the multi-agent workflow
    Includes messages, patient input, and workflow states
    """
    patient_input: str
    messages: Annotated[List[dict], add]
    diagnosis: str
    recommendations: str
    appointment_details: str

mocked_report_empty : Report = {'patient_info': '',
                          'messages' : [],
                          'diagnosis' : '',
                          'recommendations' : '',
                          'appointment_details' : '',
}

mocked_report_patient_filled : Report = {'patient_info': 'The user is a 45-year-old male named John Doe who is experiencing persistent chest pain, shortness of breath, occasional dizziness, and fatigue for the past two weeks. His medical history includes hypertension managed with medication and borderline high cholesterol with no current treatment.',
                          'messages' : [],
                          'diagnosis' : '',
                          'recommendations' : '',
                          'appointment_details' : '',
}

mocked_report_patient_unknwon : Report = {'patient_info': 'The curious fox dashed swiftly through the golden meadow, chasing shadows cast by the setting sun.',
                          'messages' : [],
                          'diagnosis' : '',
                          'recommendations' : '',
                          'appointment_details' : '',
}

mocked_report_diagnosis_filled : Report = {'patient_info': 'The user is a 45-year-old male named John Doe who is experiencing persistent chest pain, shortness of breath, occasional dizziness, and fatigue for the past two weeks. His medical history includes hypertension managed with medication and borderline high cholesterol with no current treatment.',
                          'messages' : [],
                          'diagnosis' : 'The possible diagnoses for John Doe include acute coronary syndrome, stable angina, heart failure, hypertensive heart disease, pulmonary embolism, gastroesophageal reflux disease, or anxiety/panic attack, and further tests are needed to confirm the diagnosis.',
                          'recommendations' : '',
                          'appointment_details' : '',
}

mocked_report_recommendation_filled : Report = {'patient_info': 'The user is a 45-year-old male named John Doe who is experiencing persistent chest pain, shortness of breath, occasional dizziness, and fatigue for the past two weeks. His medical history includes hypertension managed with medication and borderline high cholesterol with no current treatment.',
                          'messages' : [],
                          'diagnosis' : 'The possible diagnoses for John Doe include acute coronary syndrome, stable angina, heart failure, hypertensive heart disease, pulmonary embolism, gastroesophageal reflux disease, or anxiety/panic attack, and further tests are needed to confirm the diagnosis.',
                          'recommendations' : 'the patient should get a ECG scan immediately',
                          'appointment_details' : '',
}

mock_patient_info = """
John Doe, a 45-year-old male, reports experiencing persistent chest pain, shortness of breath, occasional dizziness, and fatigue for the past two weeks. His medical history includes hypertension diagnosed five years ago, managed with medication, and borderline high cholesterol with no current treatment. He has a family history of heart disease, as his father suffered a heart attack at age 50. He has no known allergies.
"""

pdf_directory_one = 'data\diagnosis'

#### Test Base LLMs

In [ ]:
# PASS
def test_base_llm():
    test_agent = OpenAILLMs()
    print(test_agent.prompt)
    test_agent_result = test_agent("What is the weather today")
    return test_agent_result
# print(test_base_llm())

#### Test Master Delegation

In [ ]:
# PASS : USER INTERACTION
def test_master_default():
    master_agent = MasterWorkflow()
    output = master_agent.test_call(mocked_report_empty)
    return output
# print(test_master_default())

# PASS : DIAGNOSIS
def test_master_diagnosis():
    master_agent = MasterWorkflow()
    output = master_agent.test_call(mocked_report_patient_filled)
    return output
# print(test_master_diagnosis())

# PASS : RECOMMENDATION
def test_master_recommendation():
    master_agent = MasterWorkflow()
    output = master_agent.test_call(mocked_report_diagnosis_filled)
    return output
# print(test_master_recommendation())



#### Test Nurse Agent and Tools

In [ ]:
# PASS
def test_nurse_patient_info():
    "my symptoms are shortness of breath and acute chest pain"
    "32"
    "i do not have any medical history"
    nurse_agent = NurseLLM()
    output = nurse_agent(mocked_report_empty)
    return output
# print(test_nurse_patient_info())

# PASS
def test_nurse_appointment():
    nurse_agent = NurseLLM()
    output = nurse_agent(mocked_report_recommendation_filled)
    return output
# print(test_nurse_appointment())

#### Test Diagnosis Agent and Tools

In [ ]:
# PASS
def test_diagnose_filled():
    diagnosis_agent = DiagnosisLLM()
    output = diagnosis_agent(mocked_report_patient_filled)
    return output
# print(test_diagnose_filled())

def test_diagnose_unknown():
    diagnosis_agent = DiagnosisLLM()
    output = diagnosis_agent(mocked_report_patient_unknwon)
    return output
# print(test_diagnose_unknown())

#### Test Synergies and Workflow

In [ ]:
# STEP ONE PASS : MASTERS TO USER
def master_to_user():
    master_agent = MasterWorkflow()
    output = master_agent.test_run(mocked_report_empty)
    return output
# print(master_to_user())

# STEP TWO PASS: MASTER TO USER TO DIAGNOSIS
def master_to_diagnosis():
    master_agent = MasterWorkflow()
    output = master_agent.test_run(mocked_report_patient_filled)
    return output
# print(master_to_diagnosis())

# STEP THREE : MASTER TO USER TO DIAGNOSIS TO RECOMMENDATION
def master_to_recommendation():
    return None


input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'] input_types={} partial_variables={} messages=[SystemMessage(content="\nYou are the orchestrator in a hospital setting. Your task is to manage and delegate tasks across the following agents, one at a time.\n\nNurse Agent\n   - Completes the 'patient_info' in the report.\n\nDiagnosis Agent\n   - Completes 'diagnosis' based on the 'patient_info' report.\n\nRecommendation Agent\n   - Suggests next steps after a diagnosis (e.g., treatment plans, tests, referrals).\n\nSchedule Agent\n   - Schedules appointments based on recommendations and patient's availability.\n", additional_kwargs={}, response_metadata={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], input_types={}, partial_variables={}, template="\nPlease delegate the agents needed to fill out the report below:\n\nYou do not have any tools:\n\n{tools} or {tool_names}\n\nHere are the agents names

KeyboardInterrupt: 

#### Test Utils

In [ ]:
# test vector database
def test_vector_database():
    # PASS
    vector_database = PDFVectorDatabase()
    # PASS
    vector_database.add_pdf_directory(pdf_directory_one)
    # PASS
    vector_database.load_pdfs()
    # PASS
    vector_database.build_database()
    # PASS
    output = vector_database.search_with_metadata(query="something is wrong with my cardiovascular")
    return type(output)
# test_vector_database()

# test the search engine
def test_search_engine():
    # PASS
    search_engine = DDGSearch()
    # PASS
    search_result = search_engine.search("atrial fibrillation symptoms and palpitations, fatigue, dizziness, and history of hypothyroidism")
    return search_result
# test_search_engine()


#### Main Workflow

In [ ]:
# # creates class 
# final_workflow = MasterWorkflow() # called once

# # starts with initial form
# initiate = final_workflow()